In [1]:
import pandas as pd

In [2]:
import urllib.request
from tqdm import tqdm

#https://stackoverflow.com/questions/15644964/python-progress-bar-and-downloads

class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

#### Downloading files

##### Alignment (.bam) files

In [ ]:
!mkdir bam

In [ ]:
data_source = pd.read_excel('Dataset_Identifiers.xlsx')
data_source

In [ ]:
path_A = 'https://www.encodeproject.org/files/'
path_B = '/@@download/'
path_C = '.bam'

for index, row in data_source.iterrows():
    download_url(path_A + row['id'] + path_B + row['id'] + path_C, 'bam/' + row['newid'] + path_C)

##### Human Genome (.fasta)

In [ ]:
#genome from https://www.encodeproject.org/data-standards/reference-sequences/
genome_url = 'https://www.encodeproject.org/files/GRCh38_no_alt_analysis_set_GCA_000001405.15/@@download/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta.gz'
download_url(genome_url, 'hg38.fasta.gz')

In [ ]:
!gzip -d ./hg38.fasta.gz

##### Blacklist Regions (.bed)

In [ ]:
#Black list regions from https://github.com/Boyle-Lab/Blacklist

#BLR_url = 'https://www.github.com/Boyle-Lab/Blacklist/blob/master/lists/hg38-blacklist.v2.bed.gz'
#download_url(BLR_url, 'hg38-blacklist.v2.bed.gz')

#something weird is going on, manually download this file 

In [13]:
!gzip -d hg38-blacklist.v2.bed.gz

#### Peak Calling

In [ ]:
# !for x in YY1 Input_RM; do samtools merge ./bam_merged/${x}_RepM.bam `ls ./bam/${x}*` --threads 4; done

!macs3 callpeak -t `ls ./bam/YY1*` -c `ls ./bam/Input_RM*` -f BAM -g hs -q 0.001 -n YY1 --outdir ./YY1_peaks --call-summits

#### Motif Discovery 

##### Top Peaks for Motifs 

In [ ]:
summits_columns = ['chr', 'start', 'end', 'name', 'score']
YY1_summits = pd.read_csv('YY1_peaks/' + 'YY1_summits.bed', sep='\t', header = None, names = summits_columns)

toppeaks = 3000
seqsize = 200 

YY1_summits = YY1_summits.sort_values('score', ascending = False, ignore_index = True).iloc[:toppeaks]
YY1_summits['start'] = YY1_summits['start'] - (seqsize // 2) 
YY1_summits['end'] = YY1_summits['end'] + (seqsize // 2) 

YY1_summits.to_csv('YY1_peaks/' + 'YY1_summits_toppeaks.bed', header=None, index=None, sep = '\t') 

##### Sequence extraction

In [ ]:
!bedtools getfasta -fi ./hg38.fasta -fo ./YY1_peaks/YY1_summits_toppeaks.fa -bed ./YY1_peaks/YY1_summits_toppeaks.bed -s -name+

##### MEME

In [ ]:
!meme ./YY1_peaks/YY1_summits_toppeaks.fa -o ./YY1_peaks/meme -dna -revcomp -nmotifs 20 -minw 7 -maxw 15 -mod zoops -p 8

#### Fragment Size Estimation

In [ ]:
!macs3 predictd -i `ls ./bam/YY1*` -g hs --outdir ./predictd